In [104]:
import os  
from dotenv import load_dotenv 

# Language Model (LLM) imports
from langchain_google_genai import GoogleGenerativeAIEmbeddings  # Embedding model from Google Gemini
from langchain_google_genai import ChatGoogleGenerativeAI  # Chat model from Google Gemini
from langchain_groq import ChatGroq  # Chat model from Groq (alternative to OpenAI)

# Chat memory management
from langchain_core.chat_history import BaseChatMessageHistory  # Base class for storing chat history
from langchain_core.chat_history import InMemoryChatMessageHistory  # Stores chat history in memory
from langchain_core.runnables.history import RunnableWithMessageHistory  # Enables message history in pipelines

# RAG (Retrieval-Augmented Generation) with LCEL (LangChain Expression Language)
from langchain_community.document_loaders import TextLoader  # Loads text from a single file
from langchain_community.document_loaders import DirectoryLoader  # Loads multiple files from a directory
from langchain.text_splitter import RecursiveCharacterTextSplitter  # Splits text into smaller chunks
from langchain_community.vectorstores import Chroma  # Stores embeddings and enables similarity search
# Runnables for modular pipelines
from langchain_core.runnables import RunnableParallel  # Runs multiple functions in parallel
from langchain_core.runnables import RunnablePassthrough  # Passes input unchanged
# Output parsing
from langchain_core.output_parsers import StrOutputParser  # Converts model output into a clean string
from langchain_core.prompts import PromptTemplate # Creates structured prompts for LLMs.

# Tools
from langchain_community.tools import WikipediaQueryRun  # Searches Wikipedia and retrieves summaries
from langchain_community.tools import YouTubeSearchTool  # Searches YouTube for relevant videos
from langchain_community.utilities import WikipediaAPIWrapper  # Provides an API wrapper for Wikipedia queries
from langchain_community.tools.tavily_search import TavilySearchResults  # Fetches search results from Tavily (a web search tool)

# Agents
from langchain.agents import AgentType  # Defines different types of agents (e.g., conversational, task-based)
from langchain.agents import load_tools  # Loads tools that agents can use (e.g., search engines, APIs)
from langchain.agents import initialize_agent  # Initializes an agent with tools and agent type for execution

In [24]:
load_dotenv()

True

In [ ]:
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
LANGCHAIN_PROJECT = os.getenv("LANGCHAIN_PROJECT")

NameError: name 'os' is not defined

In [ ]:
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY
os.environ["LANGCHAIN_PROJECT"] = LANGCHAIN_PROJECT
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"

In [75]:
embeddings  = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
llm_g = ChatGoogleGenerativeAI(model="gemini-1.5-pro", api_version="v1beta")

In [28]:
# embeddings  = HuggingFaceEmbeddings(model="all-MiniLM-L6-v2")
llm = ChatGroq(model="Gemma2-9b-It")

# Simple AI Assistant

In [29]:
while True:
    question = input("Type your question. Please write 'quit' if you dont want to ask.")
    if question.lower() != 'quit' :
        print(llm.invoke(question).content)
        print()
        print("If it not satisfies you, then I wont feel bad, as I'm just a 'Simple AI Assistant' ")
        print('____________________________________________________________________________________________')
    else:
        print("Too afraid to test me huh !")
        break

Too afraid to test me huh !


In [30]:
llm.invoke("Hi, I am Dhruv").content

"Hi Dhruv, it's nice to meet you!\n\nWhat can I do for you today? 😊\n"

In [31]:
llm.invoke("tell me what's my name is ?").content

"As an AI, I have no memory of past conversations and no access to personal information about you. Therefore, I don't know your name. 😊\n\nIf you'd like to tell me your name, I'd be happy to know!\n"

It will not have the context of previous chat, thats why creating a memory for it

In [32]:
store = {}

In [33]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

Can create new sessions like "New Chat" in chatGPT

In [34]:
config = {
    "configurable": {
        "session_id": "your_unique_session_id_here"
    }
}

In [35]:
llm_with_memory = RunnableWithMessageHistory(llm,get_session_history)

In [36]:
llm_with_memory.invoke(("Hi, I am Dhruv"),config=config).content

"Hi Dhruv, it's nice to meet you!\n\nWhat can I do for you today? 😊\n"

In [37]:
llm_with_memory.invoke(("tell me what's my name is ?"),config=config).content

'Your name is Dhruv, as you told me earlier!  😄  \n\nIs there anything else I can help you with?\n'

In [38]:
store

{'your_unique_session_id_here': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi, I am Dhruv', additional_kwargs={}, response_metadata={}), AIMessage(content="Hi Dhruv, it's nice to meet you!\n\nWhat can I do for you today? 😊\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 16, 'total_tokens': 42, 'completion_time': 0.047272727, 'prompt_time': 0.001928877, 'queue_time': 0.230968622, 'total_time': 0.049201604}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-7b6418c6-63ff-46ff-8134-1b87231b2da8-0', usage_metadata={'input_tokens': 16, 'output_tokens': 26, 'total_tokens': 42}), HumanMessage(content="tell me what's my name is ?", additional_kwargs={}, response_metadata={}), AIMessage(content='Your name is Dhruv, as you told me earlier!  😄  \n\nIs there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'comp

# RAG WITH LCEL

In [ ]:
# reading the source file from 'data' directory

loader = DirectoryLoader(
    '../data1',
    glob="./*.txt",
    loader_cls=TextLoader
)
docs = loader.load()

In [53]:
docs

[Document(metadata={'source': '..\\data\\llama3.txt'}, page_content='Llama (Large Language Model Meta AI) is a family of autoregressive large language models released by Meta AI starting in February 2023.[2][3] The latest version is Llama 3 released in April 2024.[4]\n\nModel weights for the first version of Llama were released to the research community under a non-commercial license.[5][3] Subsequent versions of Llama were made accessible outside academia and released under licenses that permitted some commercial use.[6][7] Llama models are trained at different parameter sizes, typically ranging between 7B and 70B.[4] Originally, Llama was only available as a foundation model.[8] Starting with Llama 2, Meta AI started releasing instruction fine-tuned versions alongside foundation models.[7]\n\nLlama models have been compared favorably against other large language models. Meta AI reported the original 13B parameter model\'s performance on most NLP benchmarks exceeded that of the much l

In [44]:
# creating chunks

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 50,
    chunk_overlap = 10,
    length_function = len
)

In [46]:
new_docs = text_splitter.split_documents(documents=docs)
doc_strings = [doc.page_content for doc in new_docs]

In [51]:
new_docs[:5]

[Document(metadata={'source': '..\\data\\llama3.txt'}, page_content='Llama (Large Language Model Meta AI) is a family'),
 Document(metadata={'source': '..\\data\\llama3.txt'}, page_content='a family of autoregressive large language models'),
 Document(metadata={'source': '..\\data\\llama3.txt'}, page_content='models released by Meta AI starting in February'),
 Document(metadata={'source': '..\\data\\llama3.txt'}, page_content='February 2023.[2][3] The latest version is Llama'),
 Document(metadata={'source': '..\\data\\llama3.txt'}, page_content='is Llama 3 released in April 2024.[4]')]

In [52]:
doc_strings[:5]

['Llama (Large Language Model Meta AI) is a family',
 'a family of autoregressive large language models',
 'models released by Meta AI starting in February',
 'February 2023.[2][3] The latest version is Llama',
 'is Llama 3 released in April 2024.[4]']

In [56]:
# creating retriever using vector DB

db = Chroma.from_documents(new_docs, embeddings)
retriever = db.as_retriever(search_kwargs={"k":4}) # Retrieves the top 4 most similar document chunks for any query.

In [58]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = PromptTemplate.from_template(template)

In [77]:
retrieval_chain = (
    RunnableParallel({"context" : retriever, "question" : RunnablePassthrough()})
    | prompt
    | llm_g
    | StrOutputParser()
)

In [78]:
question = "what is llama3? Can you highlight 3 important points"
print(retrieval_chain.invoke(question))

Based on the provided text:

1. **Llama 3 has an 8B parameter version.** This refers to the size of the model, indicating its complexity.
2. **Llama 3 was (potentially) released in April 2024.**  The repeated phrase with the citation suggests a possible release date, but the text doesn't definitively confirm it.
3. **The provided text fragments give a partial description of Llama 3 but don't fully define it.** We know some details, but not its overall purpose or capabilities.


# Tools & Agents

In [82]:
api_wrapper = WikipediaAPIWrapper()

In [89]:
tool1 = WikipediaQueryRun(api_wrapper=api_wrapper)

In [90]:
tool1.name

'wikipedia'

In [ ]:
tool1.description

'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.'

In [ ]:
tool1.args

{'query': {'description': 'query to look up on wikipedia',
  'title': 'Query',
  'type': 'string'}}

In [ ]:
print(tool1.run({"query" : "langchain"}))

Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.

Page: Retrieval-augmented generation
Summary: Retrieval-augmented generation (RAG) is a technique that enables generative artificial intelligence (Gen AI) models to retrieve and incorporate new information. It modifies interactions with a large language model (LLM) so that the model responds to user queries with reference to a specified set of documents, using this information to supplement information from its pre-existing training data. This allows LLMs to use domain-specific and/or updated information.  Use cases include providing chatbot access to internal company data or generating responses based on authoritative sources.
RAG improves lar

In [91]:
tool2 = YouTubeSearchTool()

In [92]:
tool2.name

'youtube_search'

In [93]:
tool2.description

'search for youtube videos associated with a person. the input to this tool should be a comma separated list, the first part contains a person name and the second a number that is the maximum number of video results to return aka num_results. the second part is optional'

In [94]:
tool2.args

{'query': {'title': 'Query', 'type': 'string'}}

In [96]:
tool2.run("Sidemen")

"['https://www.youtube.com/watch?v=xgwaz7Z9FIU&pp=ygUHU2lkZW1lbtIHCQm9AIO1pN6f1A%3D%3D', 'https://www.youtube.com/watch?v=B9ou3pu3xSQ&pp=ygUHU2lkZW1lbg%3D%3D']"

In [98]:
tool3 = TavilySearchResults()

In [99]:
tool3.name

'tavily_search_results_json'

In [100]:
tool3.description

'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.'

In [101]:
tool3.args

{'query': {'description': 'search query to look up',
  'title': 'Query',
  'type': 'string'}}

In [102]:
tool3.invoke({"query" : "Why is India failing to give jobs to youth?"})

[{'title': 'How India Is Failing Its Educated Youth - YouTube',
  'url': 'https://www.youtube.com/watch?v=cHkFU8BzeDw',
  'content': "How India Is Failing Its Educated Youth \n Bloomberg Originals \n 29806 likes \n 1136136 views \n 6 Sep 2024 \n India is the world's fastest growing major economy. Yet, it’s struggling to create jobs for its expanding population. According to research, unemployment for those aged 20 to 24 has risen to over 42%. It’s a key challenge for Prime Minister Narendra Modi, who hopes to achieve developed economy status for the country by 2047.",
  'score': 0.81979275},
 {'title': "Why Can't Young Indians Find Jobs? What is the employment rate in ...",
  'url': 'https://www.outlookbusiness.com/in-depth/why-cant-young-indians-find-jobs',
  'content': 'India’s jobs conundrum is a complex, hydra-headed challenge. Not only are the country’s youth desperate to get jobs, there are also questions of skills shortage, a fetish for sarkari jobs and policy bungling by succes

In [105]:
tool = load_tools(["wikipedia"],llm=llm)

In [ ]:
# ReAct (Reasoning + Acting) Agent
agent = initialize_agent(
    tool,  # List of tools the agent can use
    llm,  # The Language Model that powers the agent
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # Agent type: "Zero-Shot ReAct"
    verbose=True  # Enables detailed logging of the agent's thought process
)

In [108]:
agent.run("what is the current empolyement rate in India ?")

C:\Users\Dhruv\AppData\Local\Temp\ipykernel_11296\831658667.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent.run("what is the current empolyement rate in India ?")




> Entering new AgentExecutor chain...
Thought: I need to look up the current employment rate in India. 
Action: wikipedia
Action Input: India employment rate
Observation: Page: Unemployment in India
Summary: Statistics on unemployment in India had traditionally been collected, compiled and disseminated once every ten years by the Ministry of Labour and Employment (MLE), primarily from sample studies conducted by the National Sample Survey Office. Other than these 5-year sample studies, India had historically not collected monthly, quarterly or yearly nationwide employment and unemployment statistics on a routine basis. In 2016, the Centre for Monitoring Indian Economy, a non-governmental entity based in Mumbai, started sampling and publishing monthly unemployment in India statistics. Despite having one of the longest working hours, India has one of the lowest workforce productivity levels in the world. Economists often say that due to structural economic problems, India is experienci

'The current employment rate in India is 93.6 percent (100% - 6.4%).'